### 1. Multiple disk clutch brake

In [1]:
import numpy as np

# Constants for the problem
pi = np.pi

# Constants
mu = 0.5
s = 1.5
Ms = 40  # Nm
Mf = 3  # Nm
n = 250  # rpm
p_max = 1e6  # Pa (1 MPa)
Iz = 55e-6  # kg mm^2 to kg m^2
T_max = 15  # s
F_max = 1000  # N
r_imin = 55e-3  # m
r_omax = 110e-3  # m
delta_r = 20e-3  # delta_r = 20 mm
doh = 0.01 #Assumption

# Thickness bounds
t_max = 3e-3  # Max thickness in m
t_min = 1.5e-3  # Min thickness in m
l_max = 30e-3  # Max length in m
Z_max = 10  # Max number of disks
v_srmax = 10  # Max relative speed m/s

# Define bounds based on the existing constants
bounds = np.array([
    [r_imin, r_omax],  # r_o
    [r_imin, r_omax],  # r_i
    [t_min, t_max],    # t
    [1, Z_max],        # Z
    [1e-3, 10]         # rho (add a reasonable range for rho)
])

# Objective function
def objective(x):
    r_o, r_i, t, Z, rho = x
    return pi * (r_o**2 - r_i**2) * t * (Z + 1) * rho

# Constraint functions with checks
def constraint1(x):
    r_o, r_i, _, _, _ = x
    return r_o - r_i - delta_r

def constraint2(x):
    _, _, t, Z, _ = x
    return l_max - (Z + 1) * (t + doh)

def constraint3(x):
    r_o, r_i, _, _, _ = x
    if r_o == r_i:
        return 0  # or a very small number to indicate failure
    p_rz = F_max / (pi * (r_o**2 - r_i**2))
    return p_max - p_rz

def constraint4(x):
    r_o, r_i, _, _, _ = x
    if r_o == r_i:
        return 0  # or handle it differently as needed
    v_sr = (2 * pi * n * (r_o**3 - r_i**3)) / (90 * (r_o**2 - r_i**2))
    p_rz = F_max / (pi * (r_o**2 - r_i**2))
    return p_max * v_srmax - p_rz * v_sr

def constraint5(x):
    r_o, r_i, _, _, _ = x
    if r_o == r_i:
        return 0  # or handle it differently
    v_sr = (2 * pi * n * (r_o**3 - r_i**3)) / (90 * (r_o**2 - r_i**2))
    return v_srmax - v_sr

def constraint6(x):
    r_o, r_i, _, Z, _ = x
    if r_o == r_i:
        return 0  # or handle it differently
    M_h = (2/3) * mu * F_max * Z * (r_o**3 - r_i**3) / (r_o**2 - r_i**2)
    T = (Iz * pi * n) / (30 * (M_h + Mf))
    return T_max - T

def constraint7(x):
    r_o, r_i, _, Z, _ = x
    if r_o == r_i:
        return 0  # or handle it differently
    M_h = (2/3) * mu * F_max * Z * (r_o**3 - r_i**3) / (r_o**2 - r_i**2)
    return 1.5 * Ms - M_h

def constraint8(x):
    r_o, r_i, _, Z, _ = x
    if r_o == r_i:
        return 0  # or handle it differently
    M_h = (2/3) * mu * F_max * Z * (r_o**3 - r_i**3) / (r_o**2 - r_i**2)
    T = (Iz * pi * n) / (30 * (M_h + Mf))
    return T  # T >= 0

# Combine constraints
def evaluate_constraints(x):
    constraints = [constraint1, constraint2, constraint3, constraint4, constraint5, constraint6, constraint7, constraint8]
    return np.array([con(x) for con in constraints])

# BOA using above Objective Function

In [2]:
import numpy as np
import pandas as pd

# Example objective function (replace with your actual function)
def objective_function(x):
    # Example: r_o, r_i, t, Z, rho
    r_o, r_i, t, Z, rho = x
    # Example objective function (this needs to be adjusted to your problem)
    return np.pi * (r_o**2 - r_i**2) * t * (Z + 1) * rho

# Constraint evaluation function (penalty for constraint violations)
def penalty_function(x):
    constraints = evaluate_constraints(x)
    return np.sum(np.where(constraints < 0, -constraints, 0))  # Sum of constraint violations

# Constraint functions from your problem (with penalty handling)
def evaluate_constraints(x):
    # You need to include the actual constraints here, e.g.
    # constraint1, constraint2, ..., constraint8
    constraints = []
    constraints.append(x[0] - x[1] - 0.02)  # Example: r_o - r_i should be greater than 0.02
    constraints.append(x[2] - 0.003)  # Example: thickness should be <= 0.003
    # Add other constraints as needed
    return np.array(constraints)

# Butterfly Optimization Algorithm (BOA)
def butterfly_optimization_algorithm(num_butterflies, dim, bounds, max_iter, objective_function, penalty_function):
    loss_values = []

    # Parameters for BOA
    a = np.random.rand()  # Sensory modality
    c = 0.01  # Power exponent
    p = 0.8  # Switching probability

    # Initialize butterfly positions and fragrances
    butterflies = np.random.uniform(bounds[:, 0], bounds[:, 1], (num_butterflies, dim))

    # Evaluate initial butterfly fragrances (with penalty)
    I = np.apply_along_axis(objective_function, 1, butterflies)
    penalties = np.apply_along_axis(penalty_function, 1, butterflies)
    fragrances = a * (np.sign(I) * (np.abs(I)) ** c) + penalties + np.finfo(float).eps  # Objective + Penalty

    # Get the best butterfly (global best)
    best_butterfly = butterflies[np.argmin(fragrances)]
    best_fitness = np.min(fragrances)
    loss_values.append(best_fitness)

    # Main optimization loop
    for t in range(max_iter):
        for i in range(num_butterflies):
            r = np.random.rand()  # Random number to switch between global and local search

            # Global search (local optimum guidance)
            if r < p:
                butterflies[i] += ((np.random.rand() ** 2) * best_butterfly - butterflies[i]) * fragrances[i]
            else:  # Local search (random movement)
                j = np.random.randint(0, num_butterflies)
                k = np.random.randint(0, num_butterflies)
                butterflies[i] += ((np.random.rand() ** 2) * butterflies[j] - butterflies[k]) * fragrances[i]

            # Keep butterflies within bounds
            butterflies[i] = np.clip(butterflies[i], bounds[:, 0], bounds[:, 1])

        # Update fragrance (fitness + penalty)
        I = np.apply_along_axis(objective_function, 1, butterflies)
        penalties = np.apply_along_axis(penalty_function, 1, butterflies)
        fragrances = a * (np.sign(I) * (np.abs(I)) ** c) + penalties

        # Update global best butterfly
        min_fitness = np.min(fragrances)
        if min_fitness < best_fitness:
            best_fitness = min_fitness
            best_butterfly = butterflies[np.argmin(fragrances)]

        loss_values.append(best_fitness)

        # Update the value of a (sensory modality)
        a = np.random.rand()

        # Display the iteration and the best fitness value
        print(f"Iteration {t + 1}/{max_iter}, Best fitness: {best_fitness}")

    return best_butterfly, best_fitness, loss_values

# Example usage:
num_butterflies = 30
dim = 5  # Number of design variables: r_o, r_i, t, Z, rho
max_iter = 100
bounds = np.array([
    [0.055, 0.11],  # r_o bounds
    [0.055, 0.11],  # r_i bounds
    [0.0015, 0.003],  # thickness bounds
    [1, 10],  # Z bounds
    [1e-3, 10]  # rho bounds
])

# Run the Butterfly Optimization Algorithm
best_butterfly, best_fitness, loss_values = butterfly_optimization_algorithm(
    num_butterflies=num_butterflies,
    dim=dim,
    bounds=bounds,
    max_iter=max_iter,
    objective_function=objective_function,
    penalty_function=penalty_function
)

# Create a list to store data for each row
data = []
data.append(['Best Butterfly (Solution)'] + best_butterfly.tolist())
data.append(['Best Fitness', best_fitness])
data.append(['Loss Values'] + loss_values)

# Create DataFrame with data in rows
results_df = pd.DataFrame(data)

# Save to Excel
results_df.to_excel('optimization_results.xlsx', index=False, header=False)
print("Data saved to optimization_results.xlsx")

Iteration 1/100, Best fitness: -0.028745618485658233
Iteration 2/100, Best fitness: -0.17178461069648554
Iteration 3/100, Best fitness: -0.428811941165043
Iteration 4/100, Best fitness: -0.428811941165043
Iteration 5/100, Best fitness: -0.7129660240160653
Iteration 6/100, Best fitness: -0.7129660240160653
Iteration 7/100, Best fitness: -0.7129660240160653
Iteration 8/100, Best fitness: -0.7129660240160653
Iteration 9/100, Best fitness: -0.7129660240160653
Iteration 10/100, Best fitness: -0.7129660240160653
Iteration 11/100, Best fitness: -0.7129660240160653
Iteration 12/100, Best fitness: -0.7129660240160653
Iteration 13/100, Best fitness: -0.7129660240160653
Iteration 14/100, Best fitness: -0.7129660240160653
Iteration 15/100, Best fitness: -0.7129660240160653
Iteration 16/100, Best fitness: -0.7129660240160653
Iteration 17/100, Best fitness: -0.7129660240160653
Iteration 18/100, Best fitness: -0.7129660240160653
Iteration 19/100, Best fitness: -0.7129660240160653
Iteration 20/100, Be

### 2. Robot gripper

In [3]:
import numpy as np

# Constants for the problem
pi = np.pi
Y_min = 50
Y_max = 100
Y_G = 150
Z_max = 100
P = 100

# Variable bounds
a_min, a_max = 10, 150
b_min, b_max = 10, 150
f_min, f_max = 10, 150
c_min, c_max = 100, 200
e_min, e_max = 0, 50
l_min, l_max = 100, 300
doh_min, doh_max = 1, 3.14

# Define bounds based on the existing constants
bounds = np.array([
    [a_min, a_max],   # a
    [b_min, b_max],   # b
    [f_min, f_max],   # f
    [c_min, c_max],   # c
    [e_min, e_max],   # e
    [l_min, l_max],   # l
    [doh_min, doh_max]  # doh
])

# Helper functions for geometry calculations
def g(l, z, e):
    return np.sqrt((l - z)**2 + e**2) + 1e-10  # Small value to avoid division by zero

def alpha(a, b, l, z, e, phi):
    g_val = g(l, z, e)
    if g_val == 0 or a == 0:  # Avoid division by zero or invalid a
        return 0
    arg_alpha = (a**2 + g_val**2 - b**2) / (2 * a * g_val)
    arg_alpha_clipped = np.clip(arg_alpha, -1, 1)  # Ensure value is within [-1, 1]
    return np.arccos(arg_alpha_clipped) + phi

def beta(a, b, l, z, e, phi):
    g_val = g(l, z, e)
    if g_val == 0 or b == 0:  # Avoid division by zero or invalid b
        return 0
    arg_beta = (b**2 + g_val**2 - a**2) / (2 * b * g_val)
    arg_beta_clipped = np.clip(arg_beta, -1, 1)  # Ensure value is within [-1, 1]
    return np.arccos(arg_beta_clipped) - phi

def phi_func(e, l, z, phi_offset):
    return np.arctan(e / (l - z + 1e-10)) + phi_offset  # Avoid division by zero

# Redefined F_k function for minimization
def F_k_for_minimization(a, b, l, z, e, phi_offset, c):
    alpha_val = alpha(a, b, l, z, e, phi_offset)
    beta_val = beta(a, b, l, z, e, phi_offset)

    if np.cos(alpha_val) == 0:  # Avoid division by zero for cos(alpha)
        return 0

    return (P * b * np.sin(alpha_val + beta_val)) / (2 * c * np.cos(alpha_val))

# Objective function for minimization
def objective(x):
    a, b, f, c, e, l, doh = x
    z_min, z_max = 0, Z_max
    phi_offset = phi_func(e, l, z_min, doh)

    F_max = F_k_for_minimization(a, b, l, z_max, e, phi_offset, c)
    F_min = F_k_for_minimization(a, b, l, z_min, e, phi_offset, c)

    return F_max - F_min

# y(x, z) function
def y(x, z):
    a, b, f, c, e, l, doh = x
    phi = phi_func(e, l, z, doh)
    beta_val = beta(a, b, l, z, e, phi)
    return 2 * (e + f + c * np.sin(beta_val + doh))

# Constraint functions
def constraint1(x):
    return Y_min - y(x, Z_max)

def constraint2(x):
    return y(x, Z_max)

def constraint3(x):
    return y(x, 0) - Y_max

def constraint4(x):
    return Y_G - y(x, 0)

def constraint5(x):
    a, b, _, _, e, l, _ = x
    return (a + b)**2 - l**2 - e**2

def constraint6(x):
    a, b, _, _, e, l, _ = x
    return (l - Z_max)**2 + (a - e)**2 - b**2

def constraint7(x):
    _, _, _, _, _, l, _ = x
    return l - Z_max

# Combine constraints
def evaluate_constraints(x):
    constraints = [constraint1, constraint2, constraint3, constraint4, constraint5, constraint6, constraint7]
    return np.array([con(x) for con in constraints])

# BOA using above Objective Function

In [4]:
import numpy as np
import pandas as pd

# Penalty function for constraints (handles constraint violations)
def penalty_function(x):
    constraints = evaluate_constraints(x)
    return np.sum(np.where(constraints < 0, -constraints, 0))  # Penalty for constraint violations

# Butterfly Optimization Algorithm (BOA)
def butterfly_optimization_algorithm(num_butterflies, dim, bounds, max_iter, objective_function, penalty_function):
    loss_values = []

    # Parameters for BOA
    a = np.random.rand()  # Sensory modality
    c = 0.01  # Power exponent
    p = 0.8  # Switching probability

    # Initialize butterfly positions randomly within bounds
    butterflies = np.random.uniform(bounds[:, 0], bounds[:, 1], (num_butterflies, dim))

    # Initialize fragrances (fitness values) for butterflies
    I = np.apply_along_axis(objective_function, 1, butterflies)
    fragrances = a * (np.sign(I) * (np.abs(I)) ** c) + np.finfo(float).eps

    # Calculate penalty for constraint violations
    penalties = np.apply_along_axis(penalty_function, 1, butterflies)
    fragrances += penalties  # Add penalty to the fragrance

    # Initialize best butterfly (global best)
    best_butterfly = butterflies[np.argmin(fragrances)]
    best_fitness = np.min(fragrances)
    loss_values.append(best_fitness)

    # Main optimization loop
    for t in range(max_iter):
        for i in range(num_butterflies):
            r = np.random.rand()  # Random number to switch between global and local search

            # Global search
            if r < p:
                butterflies[i] += ((np.random.rand() ** 2) * best_butterfly - butterflies[i]) * fragrances[i]
            # Local search
            else:
                j = np.random.randint(0, num_butterflies)
                k = np.random.randint(0, num_butterflies)
                butterflies[i] += ((np.random.rand() ** 2) * butterflies[j] - butterflies[k]) * fragrances[i]

            # Ensure butterflies stay within bounds
            butterflies[i] = np.clip(butterflies[i], bounds[:, 0], bounds[:, 1])

        # Evaluate the fitness and fragrance for the updated butterflies
        I = np.apply_along_axis(objective_function, 1, butterflies)
        fragrances = a * (np.sign(I) * (np.abs(I)) ** c) + np.finfo(float).eps

        # Apply penalties for constraint violations
        penalties = np.apply_along_axis(penalty_function, 1, butterflies)
        fragrances += penalties  # Add penalty to the fragrance

        # Update global best if necessary
        min_fitness = np.min(fragrances)
        if min_fitness < best_fitness:
            best_fitness = min_fitness
            best_butterfly = butterflies[np.argmin(fragrances)]

        # Store the best fitness at each iteration
        loss_values.append(best_fitness)

        print(f"Iteration {t+1}/{max_iter}, Best fitness: {best_fitness}")

    return best_butterfly, best_fitness, loss_values

num_butterflies = 30  # Number of butterflies in the population
dim = 7  # Problem dimension (number of decision variables)
max_iter = 100  # Number of iterations

# Run BOA
best_butterfly, best_fitness, loss_values = butterfly_optimization_algorithm(
    num_butterflies, dim, bounds, max_iter, objective, penalty_function
)

results_df = pd.DataFrame({
    'Iteration': list(range(max_iter + 1)),  # Add +1 to include the initial iteration
    'Best_Fitness': loss_values,
    'Best_Butterfly': [best_butterfly] * (max_iter + 1)  # Repeat for all iterations
})
results_df.to_excel('optimization_results.xlsx', index=False)
print("Data saved to optimization_results.xlsx")


Iteration 1/100, Best fitness: 79.99999999999997
Iteration 2/100, Best fitness: 79.99999999999997
Iteration 3/100, Best fitness: 79.99999999999997
Iteration 4/100, Best fitness: 79.99999999999997
Iteration 5/100, Best fitness: 79.99999999999997
Iteration 6/100, Best fitness: 79.99999999999997
Iteration 7/100, Best fitness: 79.99999999999997
Iteration 8/100, Best fitness: 79.99999999999997
Iteration 9/100, Best fitness: 79.99999999999997
Iteration 10/100, Best fitness: 79.99999999999997
Iteration 11/100, Best fitness: 79.99999999999997
Iteration 12/100, Best fitness: 79.99999999999997
Iteration 13/100, Best fitness: 79.99999999999997
Iteration 14/100, Best fitness: 79.99999999999997
Iteration 15/100, Best fitness: 79.99999999999997
Iteration 16/100, Best fitness: 79.99999999999997
Iteration 17/100, Best fitness: 79.99999999999997
Iteration 18/100, Best fitness: 79.99999999999997
Iteration 19/100, Best fitness: 79.99999999999997
Iteration 20/100, Best fitness: 79.99999999999997
Iteration

### 3. Step-cone pulley

In [5]:
import numpy as np

# Constants for the problem
pi = np.pi

# Constants in SI units
rho = 7200  # kg/m^3 (Density)
a = 3       # m (Length)
mu = 0.35   # (Dimensionless, Friction coefficient)
s = 1.75e6  # Pa (Stress, converted from 1.75 MPa)
t = 0.008 # m
N = 100     # (Total speed for normalization, units not specified)

# Objective function
def objective(x):
    if len(x) != 9:
        raise ValueError(f"The input vector must have exactly 9 elements, but received {len(x)}.")

    # Assign each value individually
    d1, d2, d3, d4 = x[0], x[1], x[2], x[3]
    N1, N2, N3, N4 = x[4], x[5], x[6], x[7]
    w = x[8]

    # Limit values for numerical stability
    d1 = np.clip(d1, 1e-6, 1e3)
    d2 = np.clip(d2, 1e-6, 1e3)
    d3 = np.clip(d3, 1e-6, 1e3)
    d4 = np.clip(d4, 1e-6, 1e3)
    w = np.clip(w, 1e-6, 1e3)

    # Objective calculation
    term1 = d1**2 * (1 + (N1/N)**2)
    term2 = d2**2 * (1 + (N2/N)**2)
    term3 = d3**2 * (1 + (N3/N)**2)
    term4 = d4**2 * (1 + (N4/N)**2)

    # Handle overflow
    max_float = np.finfo(np.float64).max
    min_float = np.finfo(np.float64).min

    # Check for overflow and replace with max/min values
    term1 = term1 if term1 < max_float else max_float
    term2 = term2 if term2 < max_float else max_float
    term3 = term3 if term3 < max_float else max_float
    term4 = term4 if term4 < max_float else max_float

    # Check before final multiplication
    total_terms = term1 + term2 + term3 + term4
    if total_terms < max_float:
        result = rho * w * total_terms
    else:
        result = max_float  # If the total is still too high, set to max float

    return result

def constraint1(x):
    d1, d2, d3, d4 = x[0], x[1], x[2], x[3]
    N1, N2, N3, N4 = x[4], x[5], x[6], x[7]
    w = x[8]

    C1 = np.clip(np.pi * d1 / 2 * (1 + N1 / N) + ((N1/N - 1)**2 / (4 * a)) + 2 * a, -1e10, 1e10)
    C2 = np.clip(np.pi * d2 / 2 * (1 + N2 / N) + ((N2/N - 1)**2 / (4 * a)) + 2 * a, -1e10, 1e10)

    return C1 - C2

def constraint2(x):
    d1, d2, d3, d4 = x[0], x[1], x[2], x[3]
    N1, N2, N3, N4 = x[4], x[5], x[6], x[7]
    w = x[8]

    C1 = np.clip(np.pi * d1 / 2 * (1 + N1 / N) + ((N1/N - 1)**2 / (4 * a)) + 2 * a, -1e10, 1e10)
    C3 = np.clip(np.pi * d3 / 2 * (1 + N3 / N) + ((N3/N - 1)**2 / (4 * a)) + 2 * a, -1e10, 1e10)

    return C1 - C3

def constraint3(x):
    d1, d2, d3, d4 = x[0], x[1], x[2], x[3]
    N1, N2, N3, N4 = x[4], x[5], x[6], x[7]
    w = x[8]

    C1 = np.clip(np.pi * d1 / 2 * (1 + N1 / N) + ((N1/N - 1)**2 / (4 * a)) + 2 * a, -1e10, 1e10)
    C4 = np.clip(np.pi * d4 / 2 * (1 + N4 / N) + ((N4/N - 1)**2 / (4 * a)) + 2 * a, -1e10, 1e10)

    return C1 - C4

def constraint4(x):
    d1, d2, d3, d4 = x[0], x[1], x[2], x[3]
    N1, N2, N3, N4 = x[4], x[5], x[6], x[7]
    w = x[8]

    # Clip input to np.exp to avoid overflow
    R1 = np.exp(np.clip(mu * (pi - 2 * np.arcsin(np.clip((N1/N - 1) * d1 / (2 * a), -1, 1))), -700, 700))
    R2 = np.exp(np.clip(mu * (pi - 2 * np.arcsin(np.clip((N2/N - 1) * d2 / (2 * a), -1, 1))), -700, 700))
    R3 = np.exp(np.clip(mu * (pi - 2 * np.arcsin(np.clip((N3/N - 1) * d3 / (2 * a), -1, 1))), -700, 700))
    R4 = np.exp(np.clip(mu * (pi - 2 * np.arcsin(np.clip((N4/N - 1) * d4 / (2 * a), -1, 1))), -700, 700))

    return np.sum([R1, R2, R3, R4]) - 8

def constraint5(x):
    d1, d2, d3, d4 = x[0], x[1], x[2], x[3]
    N1, N2, N3, N4 = x[4], x[5], x[6], x[7]
    w = x[8]

    # Use np.clip to prevent overflow in np.exp
    P1 = s * t * w * (1 - np.exp(np.clip(-mu * (pi - 2 * np.arcsin(np.clip((N1/N - 1) * d1 / (2 * a), -1, 1))), -700, 700))) * (pi * d1 * N1 / 60)
    P2 = s * t * w * (1 - np.exp(np.clip(-mu * (pi - 2 * np.arcsin(np.clip((N2/N - 1) * d2 / (2 * a), -1, 1))), -700, 700))) * (pi * d2 * N2 / 60)
    P3 = s * t * w * (1 - np.exp(np.clip(-mu * (pi - 2 * np.arcsin(np.clip((N3/N - 1) * d3 / (2 * a), -1, 1))), -700, 700))) * (pi * d3 * N3 / 60)
    P4 = s * t * w * (1 - np.exp(np.clip(-mu * (pi - 2 * np.arcsin(np.clip((N4/N - 1) * d4 / (2 * a), -1, 1))), -700, 700))) * (pi * d4 * N4 / 60)

    return np.sum([P1, P2, P3, P4]) - (4 * 0.75 * 745.6998)

# Combine constraints
def evaluate_constraints(x):
    constraints = [constraint1, constraint2, constraint3, constraint4, constraint5]
    return np.array([con(x) for con in constraints])

# BOA using above Objective Function

In [6]:
import numpy as np
import pandas as pd

# Butterfly Optimization Algorithm (BOA)
def butterfly_optimization_algorithm(num_butterflies, dim, bounds, max_iter, objective_function, penalty_function):
    loss_values = []

    # Parameters for BOA
    a = np.random.rand()  # Sensory modality
    c = 0.01  # Power exponent
    p = 0.8  # Switching probability

    # Initialize butterfly positions within the bounds
    butterflies = np.random.uniform(bounds[:, 0], bounds[:, 1], (num_butterflies, dim))

    # Evaluate initial butterfly fragrances (with penalty)
    I = np.apply_along_axis(objective_function, 1, butterflies)
    penalties = np.apply_along_axis(penalty_function, 1, butterflies)
    fragrances = a * (np.sign(I) * (np.abs(I)) ** c) + penalties + np.finfo(float).eps  # Objective + Penalty

    # Get the best butterfly (global best)
    best_butterfly = butterflies[np.argmin(fragrances)]
    best_fitness = np.min(fragrances)
    loss_values.append(best_fitness)

    print(f"Initial Best Fitness: {best_fitness}")

    # Main optimization loop
    for t in range(max_iter):
        for i in range(num_butterflies):
            r = np.random.rand()  # Random number to switch between global and local search

            # Global search (local optimum guidance)
            if r < p:
                butterflies[i] += ((np.random.rand() ** 2) * best_butterfly - butterflies[i]) * fragrances[i]
            else:  # Local search (random movement)
                j = np.random.randint(0, num_butterflies)
                k = np.random.randint(0, num_butterflies)
                butterflies[i] += ((np.random.rand() ** 2) * butterflies[j] - butterflies[k]) * fragrances[i]

            # Keep butterflies within bounds
            butterflies[i] = np.clip(butterflies[i], bounds[:, 0], bounds[:, 1])

        # Update fragrance (fitness + penalty)
        I = np.apply_along_axis(objective_function, 1, butterflies)
        penalties = np.apply_along_axis(penalty_function, 1, butterflies)
        fragrances = a * (np.sign(I) * (np.abs(I)) ** c) + penalties

        # Update global best butterfly
        min_fitness = np.min(fragrances)
        if min_fitness < best_fitness:
            best_fitness = min_fitness
            best_butterfly = butterflies[np.argmin(fragrances)]

        loss_values.append(best_fitness)

        # Update the value of a (sensory modality)
        a = np.random.rand()

        # Display the iteration and the best fitness value
        print(f"Iteration {t + 1}/{max_iter}, Best fitness: {best_fitness}")

    return best_butterfly, best_fitness, loss_values
# Define your objective function and penalty function (replace with your own logic)
def objective_function(x):
    # Your actual objective function logic
    return np.sum(x)  # Just a placeholder example

def penalty_function(x):
    # Your constraint violation penalty logic
    penalty = 0
    if x[0] < 0 or x[1] > 10:  # Example constraint violations
        penalty += 10
    return penalty

# Set parameters
num_butterflies = 30
dim = 9  # Number of variables (e.g., d1, d2, ..., w)
max_iter = 100
bounds = np.array([
    [0.01, 1],   # Bound for d1
    [0.01, 1],   # Bound for d2
    [0.01, 1],   # Bound for d3
    [0.01, 1],   # Bound for d4
    [0.01, 100], # Bound for N1
    [0.01, 100], # Bound for N2
    [0.01, 100], # Bound for N3
    [0.01, 100], # Bound for N4
    [0.01, 10]   # Bound for w
])

# Run the algorithm
best_butterfly, best_fitness, loss_values = butterfly_optimization_algorithm(
    num_butterflies=num_butterflies,
    dim=dim,
    bounds=bounds,
    max_iter=max_iter,
    objective_function=objective_function,
    penalty_function=penalty_function
)

results_df = pd.DataFrame({
    'Iteration': list(range(max_iter + 1)),  # Add +1 to include the initial iteration
    'Best_Fitness': loss_values,
    'Best_Butterfly': [best_butterfly] * (max_iter + 1)  # Repeat for all iterations
})
results_df.to_excel('optimization_results.xlsx', index=False)
print("Data saved to optimization_results.xlsx")


Initial Best Fitness: 0.7057571097760176
Iteration 1/100, Best fitness: 0.6971787280886661
Iteration 2/100, Best fitness: 0.2931247106895947
Iteration 3/100, Best fitness: 0.2931247106895947
Iteration 4/100, Best fitness: 0.1550834058655852
Iteration 5/100, Best fitness: 0.1550834058655852
Iteration 6/100, Best fitness: 0.1550834058655852
Iteration 7/100, Best fitness: 0.1550834058655852
Iteration 8/100, Best fitness: 0.019255045735914907
Iteration 9/100, Best fitness: 0.019255045735914907
Iteration 10/100, Best fitness: 0.019255045735914907
Iteration 11/100, Best fitness: 0.019255045735914907
Iteration 12/100, Best fitness: 0.019255045735914907
Iteration 13/100, Best fitness: 0.019255045735914907
Iteration 14/100, Best fitness: 0.019255045735914907
Iteration 15/100, Best fitness: 0.019255045735914907
Iteration 16/100, Best fitness: 0.019255045735914907
Iteration 17/100, Best fitness: 0.019255045735914907
Iteration 18/100, Best fitness: 0.0037647743183483436
Iteration 19/100, Best fitn

### 4. Hydrodynamic thrust bearing design

In [7]:
import numpy as np

# Constants for the problem
pi = np.pi

# Constants
gamma = 0.0307
C = 0.5
n = -3.55
C1 = 10.04
Ws = 101000  # N
Pmax = 1000  # Pa (1 MPa)
delta_Tmax = 50  # max delta temperature
h_min = 0.001  # min height in m
g = 386.4  # acceleration due to gravity in m/s^2
N = 750  # rpm

# Define variable bounds
R_min = 1  # Minimum radius
R_max = 16  # Maximum radius
Ro_min = 1  # Minimum inner radius
Ro_max = 16  # Maximum inner radius
Q_min = 1  # Minimum flow rate
Q_max = 16  # Maximum flow rate
mu_min = 1e-6  # Minimum viscosity
mu_max = 16e-6  # Maximum viscosity

# Define bounds based on the existing constants
bounds = np.array([
    [R_min, R_max],   # R
    [Ro_min, Ro_max], # R_o
    [Q_min, Q_max],   # Q
    [mu_min, mu_max]  # mu
])

# Objective function
def objective(x):
    R, Ro, Q, mu = x
    h = calculate_h(R, Ro, Q, mu) + 1e-9  # Add a small value to avoid division by zero

    # Calculate Po and check for invalid log input
    log_input_for_po = R / (Ro + 1e-9)
    if log_input_for_po <= 0:
        Po = 0  # Set Po to 0 if the logarithm input is invalid
    else:
        Po = 6 * mu * Q / (np.pi * h**3) * np.log(log_input_for_po)  # Prevent log(0)

    # Calculate P and check for invalid log input
    log_input_for_P = 8.122e6 * mu + 0.8
    if log_input_for_P <= 0:
        P = 0  # Set P to 0 if the logarithm input is invalid
    else:
        log_input_for_P = max(log_input_for_P, 1e-10)  # Prevent log of zero
        P = (np.log10(np.log10(log_input_for_P)) - C1) / n

    delta_T = 2 * (10**P - 560)  # delta temperature
    Ef = 9336 * Q * gamma * delta_T
    return Po / 0.7 + Ef

# Height calculation based on other parameters
def calculate_h(R, Ro, Q, mu):
    log_input_for_P = 8.122e6 * mu + 0.8
    if log_input_for_P <= 0:
        P = 0  # Set P to 0 if the logarithm input is invalid
    else:
        log_input_for_P = max(log_input_for_P, 1e-10)  # Prevent log of zero
        P = (np.log10(np.log10(log_input_for_P)) - C1) / n

    delta_T = 2 * (10**P - 560)  # delta temperature
    Ef = 9336 * Q * gamma * (delta_T)  # Calculate E_f based on the new delta_T
    return (2 * pi * N / 60)**2 * (2 * pi * mu / Ef) * (R**4 / 4 - Ro**4 / 4)

# Constraint functions
def constraint1(x):
    R, Ro, Q, mu = x
    h = calculate_h(R, Ro, Q, mu) + 1e-9  # Avoid zero height

    # Calculate Po and check for invalid log input
    log_input_for_po = R / (Ro + 1e-9)
    if log_input_for_po <= 0:
        Po = 0  # Set Po to 0 if the logarithm input is invalid
    else:
        Po = 6 * mu * Q / (np.pi * h**3) * np.log(log_input_for_po)  # Avoid log(0)

    if Po == 0:
        return np.inf  # Return a very high value for infeasibility

    if R <= Ro:  # Check if R is greater than Ro
        return np.inf  # Return a very high value for infeasibility

    W = np.pi * Po / 2 * (R**2 - Ro**2) / np.log(R / (Ro + 1e-9))
    return W - Ws

def constraint2(x):
    _, _, Q, mu = x
    h = calculate_h(*x) + 1e-9  # Avoid zero height

    # Calculate Po and check for invalid log input
    log_input_for_po = x[0] / (x[1] + 1e-9)
    if log_input_for_po <= 0:
        Po = 0  # Set Po to 0 if the logarithm input is invalid
    else:
        Po = 6 * mu * Q / (np.pi * h**3) * np.log(log_input_for_po)  # Avoid log(0)

    return Pmax - Po

def constraint3(x):
    _, _, _, mu = x
    log_input_for_P = 8.122e6 * mu + 0.8
    if log_input_for_P <= 0:
        P = 0  # Set P to 0 if the logarithm input is invalid
    else:
        P = (np.log10(np.log10(log_input_for_P)) - C1) / n
    delta_T = 2 * (10**P - 560)  # delta temperature
    return delta_Tmax - delta_T

def constraint4(x):
    _, _, _, _ = x
    h = calculate_h(*x) + 1e-9  # Avoid zero height
    return h - h_min

def constraint5(x):
    R, Ro, _, _ = x
    return R - Ro

def constraint6(x):
    R, Ro, Q, mu = x
    h = calculate_h(R, Ro, Q, mu) + 1e-9  # Avoid zero height

    # Calculate Po and check for invalid log input
    log_input_for_po = R / (Ro + 1e-9)
    if log_input_for_po <= 0:
        Po = 0  # Set Po to 0 if the logarithm input is invalid
    else:
        Po = 6 * mu * Q / (np.pi * h**3) * np.log(log_input_for_po)  # Avoid log(0)

    # Check for division by zero issues
    if Po == 0 or h == 0:  # Ensure Po and h are valid
        return np.inf  # Return a very high value for infeasibility

    return 0.001 - gamma / (g * Po) * (Q / (2 * np.pi * R * h))

def constraint7(x):
    R, Ro, Q, mu = x
    h = calculate_h(R, Ro, Q, mu) + 1e-9  # Avoid zero height

    # Calculate Po and check for invalid log input
    log_input_for_po = R / (Ro + 1e-9)
    if log_input_for_po <= 0:
        Po = 0  # Set Po to 0 if the logarithm input is invalid
    else:
        Po = 6 * mu * Q / (np.pi * h**3) * np.log(log_input_for_po)  # Avoid log(0)

    if Po == 0:
        return np.inf  # Return a very high value for infeasibility

    if R <= Ro:  # Check if R is greater than Ro
        return np.inf  # Return a very high value for infeasibility

    W = np.pi * Po / 2 * (R**2 - Ro**2) / np.log(R / (Ro + 1e-9))
    return 5000 - W / (np.pi * (R**2 - Ro**2 + 1e-9))

# Combine constraints
def evaluate_constraints(x):
    constraints = [constraint1, constraint2, constraint3, constraint4, constraint5, constraint6, constraint7]
    return np.array([con(x) for con in constraints])

# BOA using above Objective Function

In [8]:
import numpy as np
import pandas as pd

# Butterfly Optimization Algorithm (BOA)
def butterfly_optimization_algorithm(num_butterflies, dim, bounds, max_iter, objective_function, penalty_function):
    # Parameters for BOA
    a = np.random.rand()  # Sensory modality
    c = 0.01  # Power exponent
    p = 0.8  # Switching probability

    # Initialize butterfly positions and fragrances
    butterflies = np.random.uniform(bounds[:, 0], bounds[:, 1], (num_butterflies, dim))
    I = np.apply_along_axis(objective_function, 1, butterflies)
    penalties = np.apply_along_axis(penalty_function, 1, butterflies)
    fragrances = a * (np.sign(I) * (np.abs(I)) ** c) + penalties + np.finfo(float).eps

    # Get the best butterfly (global best)
    best_butterfly = butterflies[np.argmin(fragrances)]
    best_fitness = np.min(fragrances)

    # Main optimization loop
    for t in range(max_iter):
        for i in range(num_butterflies):
            r = np.random.rand()  # Random number to switch between global and local search

            # Global search
            if r < p:
                butterflies[i] += ((np.random.rand() ** 2) * best_butterfly - butterflies[i]) * fragrances[i]
            else:  # Local search
                j = np.random.randint(0, num_butterflies)
                k = np.random.randint(0, num_butterflies)
                butterflies[i] += ((np.random.rand() ** 2) * butterflies[j] - butterflies[k]) * fragrances[i]

            # Keep within bounds
            butterflies[i] = np.clip(butterflies[i], bounds[:, 0], bounds[:, 1])

        # Update fragrance
        I = np.apply_along_axis(objective_function, 1, butterflies)
        penalties = np.apply_along_axis(penalty_function, 1, butterflies)
        fragrances = a * (np.sign(I) * (np.abs(I)) ** c) + penalties

        # Update global best butterfly
        min_fitness = np.min(fragrances)
        if min_fitness < best_fitness:
            best_fitness = min_fitness
            best_butterfly = butterflies[np.argmin(fragrances)]

        # Update the value of a (sensory modality)
        a = np.random.rand()

        # Display the iteration and the best fitness value
        print(f"Iteration {t + 1}/{max_iter}, Best fitness: {best_fitness:.6f}")



    return best_butterfly, best_fitness
num_butterflies = 30
dim = 4  # Number of design variables: R, Ro, Q, mu
max_iter = 100
bounds = np.array([
    [1, 16],      # R bounds
    [1, 16],      # Ro bounds
    [1, 16],      # Q bounds
    [1e-6, 16e-6] # mu bounds
])

# Call the algorithm with the problem's objective and constraints
best_butterfly, best_fitness = butterfly_optimization_algorithm(
    num_butterflies=num_butterflies,
    dim=dim,
    bounds=bounds,
    max_iter=max_iter,
    objective_function=objective,
    penalty_function=penalty_function
)

results_df = pd.DataFrame({
    'Iteration': list(range(max_iter)),
    'Best_Fitness': [best_fitness] * max_iter,
    'Best_Butterfly': [best_butterfly] * max_iter
})

results_df.to_excel('optimization_results.xlsx', index=False)
print("Data saved to optimization_results.xlsx")

Iteration 1/100, Best fitness: -1.271200
Iteration 2/100, Best fitness: -1.271200
Iteration 3/100, Best fitness: -1.271200
Iteration 4/100, Best fitness: -1.271200
Iteration 5/100, Best fitness: -1.271200
Iteration 6/100, Best fitness: -1.271200
Iteration 7/100, Best fitness: -1.271200
Iteration 8/100, Best fitness: -1.271200
Iteration 9/100, Best fitness: -1.271200
Iteration 10/100, Best fitness: -1.271200
Iteration 11/100, Best fitness: -1.271200
Iteration 12/100, Best fitness: -1.271200
Iteration 13/100, Best fitness: -1.271200
Iteration 14/100, Best fitness: -1.271200
Iteration 15/100, Best fitness: -1.271200
Iteration 16/100, Best fitness: -1.409389
Iteration 17/100, Best fitness: -1.409389
Iteration 18/100, Best fitness: -1.466379
Iteration 19/100, Best fitness: -1.466379
Iteration 20/100, Best fitness: -1.466379
Iteration 21/100, Best fitness: -1.471404
Iteration 22/100, Best fitness: -1.506300
Iteration 23/100, Best fitness: -1.506300
Iteration 24/100, Best fitness: -1.506300
I

### 5. Rolling element bearing

In [9]:
import numpy as np

# Constants for the problem
pi = np.pi

# Given constants
D = 160  # mm
d = 90   # mm
B_w = 30  # mm

# Define variable bounds
D_b_min = 0.15 * (D - d)  # 15% of (D - d)
D_b_max = 0.45 * (D - d)  # 45% of (D - d)
Z_min = 4
Z_max = 50

# Bounds for additional parameters
K_Dmin_min = 0.4
K_Dmin_max = 0.5
K_Dmax_min = 0.6
K_Dmax_max = 0.7
epsilon_min = 0.3
epsilon_max = 0.4
e_min = 0.02
e_max = 0.1
tau_min = 0.6
tau_max = 0.85

# Define bounds based on the existing constants
bounds = np.array([
    (D_b_min, D_b_max),  # D_b
    (K_Dmin_min, K_Dmin_max),  # K_Dmin
    (K_Dmax_min, K_Dmax_max),  # K_Dmax
    (Z_min, Z_max),      # Z
    (epsilon_min, epsilon_max),  # epsilon
    (e_min, e_max),      # e
    (tau_min, tau_max)   # tau
])

# Objective function
def objective(x):
    D_b, K_Dmin, K_Dmax, Z, epsilon, e, tau = x
    if D_b <= 25.4:
        C_d = f_c(D_b) * safe_power(Z, 2/3) * safe_power(D_b, 1.8)
    else:
        C_d = 3.647 * f_c(D_b) * safe_power(Z, 2/3) * safe_power(D_b, 1.4)
    return -C_d  # Minimize negative to maximize C_d

# Function to safely compute powers
def safe_power(base, exponent):
    if base < 0 and exponent % 2 == 0:  # Even exponent with a negative base
        return np.nan  # or return 0, depending on your needs
    elif base < 0:
        return np.nan  # or return a default value
    elif base == 0 and exponent < 0:
        return np.nan  # or return a default value
    return base ** exponent

# Function to calculate f_c
def f_c(D_b):
    gamma = D_b * np.cos(np.radians(30)) / (D_b + 90)  # Example angle of 30 degrees
    f_i = 0.515  # Fixed value for this example
    f_o = 0.515  # Fixed value for this example

    # Check if gamma is in the valid range
    if not (1 + gamma > 0 and 1 - gamma > 0):
        return 0  # Or set to a suitable fallback value

    # Calculate f_c value while ensuring gamma is within a valid range
    f_c_value = 37.91 * (1 + (1.04 * ((1 - gamma) / (1 + gamma))**1.72 * (f_i * (2 * f_o - 1) / (f_o * (2 * f_i - 1)))**0.41)**(10/3))**(-0.3)

    # Check if f_c_value is NaN
    if np.isnan(f_c_value):
        return 0  # Set to a fallback value if invalid

    return f_c_value

# Constraint functions
def constraint1(x):
    D_b, K_Dmin, K_Dmax, Z, epsilon, e, tau = x
    return (2 * np.arcsin(D_b / (D - d)) / (D_b / (D - d))) - Z + 1  # g1 >= 0

def constraint2(x):
    D_b, K_Dmin, K_Dmax, Z, epsilon, e, tau = x
    return 2 * D_b - K_Dmin * (D - d)  # g2 >= 0

def constraint3(x):
    D_b, K_Dmin, K_Dmax, Z, epsilon, e, tau = x
    return K_Dmax * (D - d) - 2 * D_b  # g3 >= 0

def constraint4(x):
    D_b, K_Dmin, K_Dmax, Z, epsilon, e, tau = x
    return -(tau * B_w - D_b)  # g4 <= 0

def constraint5(x):
    D_b, K_Dmin, K_Dmax, Z, epsilon, e, tau = x
    return (0.5 * (D + d)) - (D - d)  # g5 >= 0

def constraint6(x):
    D_b, K_Dmin, K_Dmax, Z, epsilon, e, tau = x
    return (0.5 + e) * (D + d) - (D - d)  # g6 >= 0

def constraint7(x):
    D_b, K_Dmin, K_Dmax, Z, epsilon, e, tau = x
    return 0.5 * (D - (D - d) - D_b) - epsilon * D_b  # g7 >= 0

def constraint8(x):
    f_i = x[4]  # f_i
    return f_i - 0.515  # g8 >= 0

def constraint9(x):
    f_o = x[5]  # f_o
    return f_o - 0.515  # g9 >= 0

# Combine constraints
def evaluate_constraints(x):
    constraints = [
        constraint1,
        constraint2,
        constraint3,
        constraint4,
        constraint5,
        constraint6,
        constraint7,
        constraint8,
        constraint9
    ]
    return np.array([con(x) for con in constraints])

# BOA using above Objective Function

In [10]:
import numpy as np
import pandas as pd

# Penalty function for constraint violations
def penalty_function(x):
    constraints = evaluate_constraints(x)
    return np.sum(np.where(constraints < 0, -constraints, 0))  # Sum of constraint violations

# Butterfly Optimization Algorithm (BOA)
def butterfly_optimization_algorithm(num_butterflies, dim, bounds, max_iter, objective, penalty_function):
    loss_values = []

    # Parameters for BOA
    a = np.random.rand()  # Sensory modality
    c = 0.01  # Power exponent
    p = 0.8  # Switching probability

    # Initialize butterfly positions and fragrances
    butterflies = np.random.uniform(bounds[:, 0], bounds[:, 1], (num_butterflies, dim))

    # Evaluate initial butterfly fragrances (with penalty)
    I = np.apply_along_axis(objective, 1, butterflies)
    penalties = np.apply_along_axis(penalty_function, 1, butterflies)
    fragrances = a * (np.sign(I) * (np.abs(I)) ** c) + penalties + np.finfo(float).eps  # Objective + Penalty

    # Get the best butterfly (global best)
    best_butterfly = butterflies[np.argmin(fragrances)]
    best_fitness = np.min(fragrances)
    loss_values.append(best_fitness)

    # Main optimization loop
    for t in range(1, max_iter + 1):
        for i in range(num_butterflies):
            r = np.random.rand()  # Random number to switch between global and local search

            # Global search (local optimum guidance)
            if r < p:
                butterflies[i] += ((np.random.rand() ** 2) * best_butterfly - butterflies[i]) * fragrances[i]
            else:  # Local search (random movement)
                j = np.random.randint(0, num_butterflies)
                k = np.random.randint(0, num_butterflies)
                butterflies[i] += ((np.random.rand() ** 2) * butterflies[j] - butterflies[k]) * fragrances[i]

            # Keep butterflies within bounds
            butterflies[i] = np.clip(butterflies[i], bounds[:, 0], bounds[:, 1])

        # Update fragrance (fitness + penalty)
        I = np.apply_along_axis(objective, 1, butterflies)
        penalties = np.apply_along_axis(penalty_function, 1, butterflies)
        fragrances = a * (np.sign(I) * (np.abs(I)) ** c) + penalties

        # Update global best butterfly
        min_fitness = np.min(fragrances)
        if min_fitness < best_fitness:
            best_fitness = min_fitness
            best_butterfly = butterflies[np.argmin(fragrances)]

        loss_values.append(best_fitness)

        # Update the value of a (sensory modality)
        a = np.random.rand()

        # Display iteration-wise progress
        print(f"Iteration {t}/{max_iter}, Best fitness: {best_fitness}")

    # Display the final output after optimization is complete
    print("\nOptimization complete!")
    print("Best butterfly (solution):", best_butterfly)
    print("Best fitness:", best_fitness)

    return best_butterfly, best_fitness, loss_values

# Example usage:
num_butterflies = 30
dim = 7  # Number of design variables as per your problem
max_iter = 100
bounds = np.array([
    [D_b_min, D_b_max],  # D_b bounds
    [K_Dmin_min, K_Dmin_max],  # K_Dmin bounds
    [K_Dmax_min, K_Dmax_max],  # K_Dmax bounds
    [Z_min, Z_max],  # Z bounds
    [epsilon_min, epsilon_max],  # epsilon bounds
    [e_min, e_max],  # e bounds
    [tau_min, tau_max]  # tau bounds
])

# Run the Butterfly Optimization Algorithm
best_butterfly, best_fitness, loss_values = butterfly_optimization_algorithm(
    num_butterflies=num_butterflies,
    dim=dim,
    bounds=bounds,
    max_iter=max_iter,
    objective=objective,
    penalty_function=penalty_function
)

results_data = []
for i in range(len(loss_values)):
    results_data.append([i + 1, loss_values[i], best_butterfly])  # Iteration, Best Fitness, Best Butterfly

results_df = pd.DataFrame(results_data, columns=['Iteration', 'Best_Fitness', 'Best_Butterfly'])
results_df.to_excel('optimization_results.xlsx', index=False)
print("Data saved to optimization_results.xlsx")

Iteration 1/100, Best fitness: 7.121628427359927
Iteration 2/100, Best fitness: 7.121628427359927
Iteration 3/100, Best fitness: 7.121628427359927
Iteration 4/100, Best fitness: 7.121628427359927
Iteration 5/100, Best fitness: 7.121628427359927
Iteration 6/100, Best fitness: 7.121628427359927
Iteration 7/100, Best fitness: 7.121628427359927
Iteration 8/100, Best fitness: 7.121628427359927
Iteration 9/100, Best fitness: 7.121628427359927
Iteration 10/100, Best fitness: 7.121628427359927
Iteration 11/100, Best fitness: 7.121628427359927
Iteration 12/100, Best fitness: 7.121628427359927
Iteration 13/100, Best fitness: 7.121628427359927
Iteration 14/100, Best fitness: 7.121628427359927
Iteration 15/100, Best fitness: 7.121628427359927
Iteration 16/100, Best fitness: 7.121628427359927
Iteration 17/100, Best fitness: 7.121628427359927
Iteration 18/100, Best fitness: 7.121628427359927
Iteration 19/100, Best fitness: 7.121628427359927
Iteration 20/100, Best fitness: 7.121628427359927
Iteration

### 6. Belleville spring

In [11]:
import numpy as np

# Define constants for the problem
pi = np.pi
eps = 1e-8  # Small epsilon to avoid division by zero

# Given constants in SI units
P_max = 23912.4  # N
doh_max = 0.00508  # m
S = 1378950000  # Pa
E = 30e6  # Pa
mu = 0.3
H = 0.0508  # m
D_max = 0.305  # m

# Define variable bounds in SI units
D_e_min = 0.0254  # m (minimum outer diameter)
D_e_max = D_max  # m (maximum outer diameter)
D_i_min = 0.00254  # m (minimum inner diameter)
D_i_max = D_max  # m (maximum inner diameter)
t_min = 0.000254  # m (minimum thickness)
t_max = 0.0254  # m (maximum thickness)
h_min = 0.000254  # m (minimum height)
h_max = H  # m (maximum height)

# Define bounds based on the existing constants
bounds = np.array([
    (D_e_min, D_e_max),  # D_e
    (D_i_min, D_i_max),  # D_i
    (t_min, t_max),      # t
    (h_min, h_max)       # h
])

# Objective function
def objective(x):
    D_e, D_i, t, h = x
    return 0.07075 * pi * (D_e**2 - D_i**2) * t

# Constraint functions
def constraint1(x):
    D_e, D_i, t, h = x
    if D_i + eps <= 0:
        return np.inf  # Return high value if inner diameter is invalid
    K = D_e / (D_i + eps)  # Prevent division by zero
    if K <= 1:  # Check if K is valid for log calculation
        return np.inf  # Return high value for infeasibility

    alpha = (6 / pi * np.log(K + eps)) * ((K - 1) / (K + eps))**2
    beta = (6 / pi * np.log(K + eps)) * (((K - 1) / (np.log(K + eps))) - 1)
    gamma = (6 / pi * np.log(K + eps)) * ((K - 1) / 2)
    return S - (4 * E * doh_max / ((1 - mu**2) * alpha * D_e**2)) * (beta * (h - (doh_max / 2)) + gamma * t)  # g1 >= 0

def constraint2(x):
    D_e, D_i, t, h = x
    if D_i + eps <= 0:
        return np.inf  # Return high value if inner diameter is invalid
    K = D_e / (D_i + eps)  # Prevent division by zero
    if K <= 1:  # Check if K is valid for log calculation
        return np.inf  # Return high value for infeasibility

    alpha = (6 / pi * np.log(K + eps)) * ((K - 1) / (K + eps))**2
    return (4 * E * doh_max / ((1 - mu**2) * alpha * D_e**2)) * ((h - (doh_max / 2)) * (h - doh_max) * t + t**3) - P_max  # g2 >= 0

def constraint3(x):
    D_e, D_i, t, h = x
    a = h / (t + eps)  # Prevent division by zero
    return objective(x) * a - doh_max  # g3 >= 0

def constraint4(x):
    D_e, D_i, t, h = x
    return H - h - t  # g4 >= 0

def constraint5(x):
    D_e, D_i, t, h = x
    return D_max - D_e  # g5 >= 0

def constraint6(x):
    D_e, D_i, t, h = x
    return D_e - D_i  # g6 >= 0

def constraint7(x):
    D_e, D_i, t, h = x
    return 0.3 - h / ((D_e - D_i) + eps)  # g7 >= 0

# Combine constraints
def evaluate_constraints(x):
    return np.array([
        constraint1(x),
        constraint2(x),
        constraint3(x),
        constraint4(x),
        constraint5(x),
        constraint6(x),
        constraint7(x)
    ])

# BOA using above Objective Function

In [12]:
import numpy as np
import pandas as pd

# Objective function
def objective_function(x):
    D_e, D_i, t, h = x
    return 0.07075 * np.pi * (D_e**2 - D_i**2) * t

# Constraint evaluation function (penalty for constraint violations)
def penalty_function(x):
    constraints = evaluate_constraints(x)
    return np.sum(np.where(constraints < 0, -constraints, 0))  # Sum of constraint violations

# Constraint functions from the given problem (with penalty handling)
def evaluate_constraints(x):
    constraints = [
        constraint1(x),
        constraint2(x),
        constraint3(x),
        constraint4(x),
        constraint5(x),
        constraint6(x),
        constraint7(x)
    ]
    return np.array(constraints)

# Butterfly Optimization Algorithm (BOA)
def butterfly_optimization_algorithm(num_butterflies, dim, bounds, max_iter, objective_function, penalty_function):
    loss_values = []

    # Parameters for BOA
    a = np.random.rand()  # Sensory modality
    c = 0.01  # Power exponent
    p = 0.8  # Switching probability

    # Initialize butterfly positions and fragrances
    butterflies = np.random.uniform(bounds[:, 0], bounds[:, 1], (num_butterflies, dim))

    # Evaluate initial butterfly fragrances (with penalty)
    I = np.apply_along_axis(objective_function, 1, butterflies)
    penalties = np.apply_along_axis(penalty_function, 1, butterflies)
    fragrances = a * (np.sign(I) * (np.abs(I)) ** c) + penalties + np.finfo(float).eps  # Objective + Penalty

    # Get the best butterfly (global best)
    best_butterfly = butterflies[np.argmin(fragrances)]
    best_fitness = np.min(fragrances)
    loss_values.append(best_fitness)

    # Main optimization loop
    for t in range(max_iter):
        for i in range(num_butterflies):
            r = np.random.rand()  # Random number to switch between global and local search

            # Global search (local optimum guidance)
            if r < p:
                butterflies[i] += ((np.random.rand() ** 2) * best_butterfly - butterflies[i]) * fragrances[i]
            else:  # Local search (random movement)
                j = np.random.randint(0, num_butterflies)
                k = np.random.randint(0, num_butterflies)
                butterflies[i] += ((np.random.rand() ** 2) * butterflies[j] - butterflies[k]) * fragrances[i]

            # Keep butterflies within bounds
            butterflies[i] = np.clip(butterflies[i], bounds[:, 0], bounds[:, 1])

        # Update fragrance (fitness + penalty)
        I = np.apply_along_axis(objective_function, 1, butterflies)
        penalties = np.apply_along_axis(penalty_function, 1, butterflies)
        fragrances = a * (np.sign(I) * (np.abs(I)) ** c) + penalties

        # Update global best butterfly
        min_fitness = np.min(fragrances)
        if min_fitness < best_fitness:
            best_fitness = min_fitness
            best_butterfly = butterflies[np.argmin(fragrances)]

        loss_values.append(best_fitness)

        # Update the value of a (sensory modality)
        a = np.random.rand()

        # Display the iteration and the best fitness value
        print(f"Iteration {t + 1}/{max_iter}, Best fitness: {best_fitness}")

    return best_butterfly, best_fitness, loss_values

# Example usage:
num_butterflies = 30
dim = 4  # Number of design variables: D_e, D_i, t, h
max_iter = 100
bounds = np.array([
    [D_e_min, D_e_max],  # D_e bounds
    [D_i_min, D_i_max],  # D_i bounds
    [t_min, t_max],      # t bounds
    [h_min, h_max]       # h bounds
])

# Run the Butterfly Optimization Algorithm
best_butterfly, best_fitness, loss_values = butterfly_optimization_algorithm(
    num_butterflies=num_butterflies,
    dim=dim,
    bounds=bounds,
    max_iter=max_iter,
    objective_function=objective_function,
    penalty_function=penalty_function
)

# Create a DataFrame
results_df = pd.DataFrame({
    'Iteration': list(range(max_iter + 1)),  # Add +1 to include the initial iteration
    'Best_Fitness': loss_values,
    'Best_Butterfly': [best_butterfly] * (max_iter + 1)  # Repeat for all iterations
})

# Save to Excel
results_df.to_excel('optimization_results.xlsx', index=False)
print("Data saved to optimization_results.xlsx")

Iteration 1/100, Best fitness: -0.8445647215607838
Iteration 2/100, Best fitness: -0.8445647215607838
Iteration 3/100, Best fitness: -0.8528782731351957
Iteration 4/100, Best fitness: -0.8528782731351957
Iteration 5/100, Best fitness: -0.8528782731351957
Iteration 6/100, Best fitness: -0.8528782731351957
Iteration 7/100, Best fitness: -0.8528782731351957
Iteration 8/100, Best fitness: -0.8528782731351957
Iteration 9/100, Best fitness: -0.8528782731351957
Iteration 10/100, Best fitness: -0.8528782731351957
Iteration 11/100, Best fitness: -0.8528782731351957
Iteration 12/100, Best fitness: -0.8528782731351957
Iteration 13/100, Best fitness: -0.8528782731351957
Iteration 14/100, Best fitness: -0.8528782731351957
Iteration 15/100, Best fitness: -0.8528782731351957
Iteration 16/100, Best fitness: -0.8528782731351957
Iteration 17/100, Best fitness: -0.8528782731351957
Iteration 18/100, Best fitness: -0.8528782731351957
Iteration 19/100, Best fitness: -0.8528782731351957
Iteration 20/100, Bes

In [ ]:
fitness_history.to_csv('PSO.csv', index=False)

NameError: name 'fitness_history' is not defined